In [1]:
# Set important paths here...
# To use Greg's file loaders and such, you must have the
# NMEG_utils directory and tell python where it is
import sys
sys.path.append( '../NMEG_utils/py_modules/' )

# Then load the needed libraries
#%matplotlib inline
import load_nmeg as ld
import matplotlib.pyplot as plt
import pandas as pd
#import seaborn as sns
import datetime as dt
#import ipdb as ipdb

In [15]:
# Years to load
start = 2010
end = 2014
# Sites to load
site = 'GLand'

# Tell python where to find the soil_qc and qc_rbd files
#data_path = 'C:/Research_Flux_Towers/Flux_Tower_Data_by_Site/' + site + '/processed_soil/'
data_path = '/home/greg/sftp/eddyflux/Soil_files/provisional/'

# Empty dict for hourly dataframes
hourly = dict()

# Fill dict with multiyear dataframes for each site in sites
def dparser( y, m, d, H, M, S ):
    yr = int( y )
    mon = int( m )
    day = int( d )
    hr = int( H )
    mn =  int( M )
    sec = int( S )
    
    # Some files have a line of zeros at the end - hack it
    #if yr != year_arg:
    #    yr = 1955
    return ( dt.datetime( yr, mon, day, hr, mn, sec ))

qc_dat = pd.DataFrame()
rbd_dat = pd.DataFrame();
for i in range( start, end+1) :
    path_qc = data_path + site + '_' + str( i ) + '_soilmet_qc.txt'
    path_rbd = data_path + site + '_' + str( i ) + '_soilmet_qc_rbd.txt'
    new_qc = pd.read_csv(path_qc, delimiter=',', parse_dates={'tstamp':[0, 1, 2, 3, 4, 5]},
                                 date_parser=dparser, index_col='tstamp')
    new_rbd = pd.read_csv(path_rbd, delimiter=',', parse_dates={'tstamp':[0, 1, 2, 3, 4, 5]},
                                 date_parser=dparser, index_col='tstamp')
    qc_dat = qc_dat.append( new_qc )
    rbd_dat = rbd_dat.append( new_rbd )
    
rbd_dat.index

DatetimeIndex(['2010-01-01 00:30:00', '2010-01-01 01:00:00',
               '2010-01-01 01:30:00', '2010-01-01 02:00:00',
               '2010-01-01 02:30:00', '2010-01-01 03:00:00',
               '2010-01-01 03:30:00', '2010-01-01 04:00:00',
               '2010-01-01 04:30:00', '2010-01-01 05:00:00', 
               ...
               '2014-12-31 19:30:00', '2014-12-31 20:00:00',
               '2014-12-31 20:30:00', '2014-12-31 21:00:00',
               '2014-12-31 21:30:00', '2014-12-31 22:00:00',
               '2014-12-31 22:30:00', '2014-12-31 23:00:00',
               '2014-12-31 23:30:00', '2015-01-01 00:00:00'],
              dtype='datetime64[ns]', name='tstamp', length=87648, freq=None, tz=None)

# get the soil columns

In [16]:
# This is where you select either SWC or SOILT columns (or others)
qc_cols = [s for s in qc_dat.columns if 'SOILT' in s and 'tcor' not in s]
rbd_cols = [s for s in rbd_dat.columns if 'SOILT' in s and 'tcor' not in s]
print(qc_cols)
print(rbd_cols)

['SOILT_AVG', 'SOILT_G1_12p5_AVG', 'SOILT_G1_22p5_AVG', 'SOILT_G1_2p5_AVG', 'SOILT_G1_37p5_AVG', 'SOILT_G1_52p5_AVG', 'SOILT_G2_12p5_AVG', 'SOILT_G2_22p5_AVG', 'SOILT_G2_2p5_AVG', 'SOILT_G2_37p5_AVG', 'SOILT_G2_52p5_AVG', 'SOILT_G3_12p5_AVG', 'SOILT_G3_22p5_AVG', 'SOILT_G3_2p5_AVG', 'SOILT_G3_37p5_AVG', 'SOILT_O1_12p5_AVG', 'SOILT_O1_22p5_AVG', 'SOILT_O1_2p5_AVG', 'SOILT_O1_37p5_AVG', 'SOILT_O1_52p5_AVG', 'SOILT_O2_12p5_AVG', 'SOILT_O2_22p5_AVG', 'SOILT_O2_2p5_AVG', 'SOILT_O2_37p5_AVG', 'SOILT_O2_52p5_AVG', 'SOILT_O3_12p5_AVG', 'SOILT_O3_22p5_AVG', 'SOILT_O3_2p5_AVG', 'SOILT_O3_37p5_AVG']
['SOILT_G1_12p5_AVG', 'SOILT_G1_22p5_AVG', 'SOILT_G1_2p5_AVG', 'SOILT_G1_37p5_AVG', 'SOILT_G1_52p5_AVG', 'SOILT_G2_12p5_AVG', 'SOILT_G2_22p5_AVG', 'SOILT_G2_2p5_AVG', 'SOILT_G2_37p5_AVG', 'SOILT_G2_52p5_AVG', 'SOILT_G3_12p5_AVG', 'SOILT_G3_22p5_AVG', 'SOILT_G3_2p5_AVG', 'SOILT_G3_37p5_AVG', 'SOILT_O1_12p5_AVG', 'SOILT_O1_22p5_AVG', 'SOILT_O1_2p5_AVG', 'SOILT_O1_37p5_AVG', 'SOILT_O1_52p5_AVG', 'SOILT_O

In [ ]:
# Here you can further reduce the number of columns to look at
# by depth or by pit

selection = 'O1' # '2p5'
sel_cols = [s for s in qc_cols if selection in s]
qc_subset = qc_dat[sel_cols]
sel_cols = [s for s in rbd_cols if selection in s]
rbd_subset = rbd_dat[sel_cols]

xlims = [ dt.datetime( start, 1, 1 ), dt.datetime( end, 12, 31 )]
ylims = [-20, 50]

fig1 = plt.figure(figsize=( 16.5, 7.5 ), dpi=150, facecolor='w', edgecolor='k')
for i, colname in enumerate( qc_subset.columns ):
    nsensors = len( qc_subset.columns )
    ax = fig1.add_subplot( nsensors, 1, i+1, sharex=ax )
    plt.plot( qc_subset.index, qc_subset[ colname ], '.r', ms=4 )
    if colname in rbd_subset.columns:
        plt.plot( rbd_subset.index, rbd_subset[ colname ], '.k', ms=4 )
    plt.ylim( ylims )
    plt.xlim( xlims )
    plt.text( dt.datetime( start, 2, 1 ), ylims[0]+.3, colname, size=14  )
    if i<nsensors-1:
        plt.setp( ax.get_xticklabels(), visible=False)
    

fig1.savefig('figures/labmeeting/GLand_qc_rbd_ex.png', dpi=300)
    
rbd_subset.plot(figsize=( 16.5, 7.5 ))
#plt.ylim( ylims )
#plt.xlim( xlims )
plt.ylabel('x')


plt.show()